In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

PREMIUMS_OUT_PATH = "Files/source/premiums_json"

# Controls how many payments to generate (keeps dataset reasonable)
MAX_MONTHS_PER_POLICY = 24   # cap long policies for a portfolio dataset (adjust later)
SEED = 42


StatementMeta(, b4ac7a1a-a378-498e-890a-d8fcda6befdf, 3, Finished, Available, Finished)

In [2]:
policies_path = "Files/source/policies_parquet"

policies = spark.read.parquet(policies_path).select(
    "policy_id", "customer_id", "agent_id",
    "policy_type", "plan_name", "premium_frequency",
    "start_date", "end_date", "policy_status",
    "sum_insured", "deductible"
)

display(policies)
print("Policies:", policies.count())

StatementMeta(, b4ac7a1a-a378-498e-890a-d8fcda6befdf, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 708d685b-35ef-4d1f-b21d-7aeaa2f743d7)

Policies: 120000


In [3]:
# Compute months between start/end, cap to keep dataset manageable
pol = (
    policies
    .withColumn("start_date", F.to_date("start_date"))
    .withColumn("end_date", F.to_date("end_date"))
    .withColumn("months_total", F.floor(F.months_between("end_date", "start_date")).cast("int"))
    .withColumn("months_total", F.when(F.col("months_total") < 1, F.lit(1)).otherwise(F.col("months_total")))
    .withColumn("months_total", F.least(F.col("months_total"), F.lit(MAX_MONTHS_PER_POLICY)))
)

# Expand rows using Spark sequence + explode
# Monthly: generate monthly dates
monthly = (
    pol.filter(F.col("premium_frequency") == "Monthly")
    .withColumn("pay_dates", F.expr("sequence(start_date, add_months(start_date, months_total), interval 1 month)"))
    .withColumn("payment_date", F.explode("pay_dates"))
    .drop("pay_dates")
)

# Annual: generate yearly dates (every 12 months)
annual = (
    pol.filter(F.col("premium_frequency") == "Annual")
    .withColumn("years_total", F.floor(F.col("months_total")/12).cast("int"))
    .withColumn("years_total", F.when(F.col("years_total") < 1, F.lit(1)).otherwise(F.col("years_total")))
    .withColumn("pay_dates", F.expr("sequence(start_date, add_months(start_date, years_total * 12), interval 12 months)"))
    .withColumn("payment_date", F.explode("pay_dates"))
    .drop("pay_dates", "years_total")
)

payments_sched = monthly.unionByName(annual)
display(payments_sched.select("policy_id","premium_frequency","start_date","end_date","months_total","payment_date"))

StatementMeta(, b4ac7a1a-a378-498e-890a-d8fcda6befdf, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 733124ba-47b9-4256-8d83-9fbb4ebcf554)

In [4]:
# Base rate by policy type (very simple)
base_rate = (
    F.when(F.col("policy_type") == "Auto",   F.lit(0.015))
     .when(F.col("policy_type") == "Health", F.lit(0.020))
     .when(F.col("policy_type") == "Home",   F.lit(0.010))
     .otherwise(F.lit(0.008))  # Life
)

# Plan multiplier
plan_mult = (
    F.when(F.col("plan_name") == "Basic",    F.lit(0.9))
     .when(F.col("plan_name") == "Standard", F.lit(1.0))
     .otherwise(F.lit(1.2))  # Premium
)

# Convert annualized premium; then split monthly if needed
annual_premium = (F.col("sum_insured") * base_rate * plan_mult)

premium_amount = (
    F.when(F.col("premium_frequency") == "Monthly", annual_premium / 12.0)
     .otherwise(annual_premium)
)

# Payment status distribution (mostly Paid)
status = (
    F.when(F.rand(SEED) < 0.92, F.lit("Paid"))
     .when(F.rand(SEED) < 0.97, F.lit("Pending"))
     .otherwise(F.lit("Failed"))
)

payments = (
    payments_sched
    .withColumn("payment_id", F.concat(F.lit("PMT-"), F.sha2(F.concat_ws("|", F.col("policy_id"), F.col("payment_date").cast("string"), F.rand(SEED).cast("string")), 256)).substr(1, 18))
    .withColumn("premium_amount", F.round(premium_amount, 2))
    .withColumn("payment_status", status)
    .withColumn("payment_method",
                F.when(F.rand(SEED) < 0.6, F.lit("BankTransfer"))
                 .when(F.rand(SEED) < 0.9, F.lit("Card"))
                 .otherwise(F.lit("Cash")))
    .withColumn("currency", F.lit("CAD"))
    .withColumn("event_ts", F.current_timestamp())
)

display(payments.select("payment_id","policy_id","payment_date","premium_amount","payment_status","payment_method"))
print("Payments rows:", payments.count())

StatementMeta(, b4ac7a1a-a378-498e-890a-d8fcda6befdf, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7d020d8c-c05f-415e-b39a-43815460b105)

Payments rows: 1522899


In [5]:
(payments
 .repartition(8)
 .write.mode("overwrite")
 .json(PREMIUMS_OUT_PATH))

print(f"✅ Premium payments JSON written to: {PREMIUMS_OUT_PATH}")

StatementMeta(, b4ac7a1a-a378-498e-890a-d8fcda6befdf, 7, Finished, Available, Finished)

✅ Premium payments JSON written to: Files/source/premiums_json


In [6]:
display(payments.groupBy("payment_status").count())
display(payments.groupBy("premium_frequency").count())

StatementMeta(, b4ac7a1a-a378-498e-890a-d8fcda6befdf, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2a468b23-2df2-4e74-a348-8bdcf3781ded)

SynapseWidget(Synapse.DataFrame, ce14ed64-89c9-4d68-a9aa-bff11da02f3d)